# Tiền xử lý dữ liệu

## Import thư viện và load data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

pd.options.display.float_format = '{:,.2f}'.format

In [2]:
df = pd.read_csv('../data/players_data.csv')
print(f'Kích thước dữ liệu: {df.shape}')
df.head(5)

Kích thước dữ liệu: (32601, 20)


,name,age_at_last_season,country_of_birth,country_of_citizenship,position,sub_position,foot,height_in_cm,current_club_name,current_club_domestic_competition_id,club_position,last_season,contract_expiration_date,market_value_in_eur,agent_name,total_goals,total_assists,total_minutes_played,total_yellow_cards,total_red_cards
0,Miroslav Klose,37.00,Poland,Germany,Attack,Centre-Forward,right,184.00,Società Sportiva Lazio S.p.A.,serie-a,8.00,2015,NaN,"1,000,000.00",ASBW Sport Marketing,12,8,2429,6,0
1,Roman Weidenfeller,37.00,Germany,Germany,Goalkeeper,Goalkeeper,left,190.00,Borussia Dortmund,bundesliga,4.00,2017,NaN,"750,000.00",Neubauer 13 GmbH,0,0,181,0,0
2,Dimitar Berbatov,34.00,Bulgaria,Bulgaria,Attack,Centre-Forward,NaN,NaN,Panthessalonikios Athlitikos Omilos Konstantin...,super-league-1,NaN,2015,NaN,"1,000,000.00",CSKA-AS-23 Ltd.,6,0,1656,0,0
3,Lúcio,34.00,Brazil,Brazil,Defender,Centre-Back,NaN,NaN,Juventus Football Club,serie-a,1.00,2012,NaN,"200,000.00",NaN,0,0,307,0,0
4,Tom Starke,36.00,East Germany (GDR),Germany,Goalkeeper,Goalkeeper,right,194.00,FC Bayern München,bundesliga,NaN,2017,NaN,"100,000.00",IFM,0,0,450,0,0


## Data cleaning
Đây là bước lọc bỏ "rác" và dữ liệu không hợp lệ.
1. Xử lý giá trị rỗng lạ: Thay thế các ký tự như "?", "N/A", "Unknown" về dạng chuẩn NaN của numpy để dễ xử lý.
2. Xóa dòng thiếu Target (market_value_in_eur):
Đây là bài toán Supervised Learning (Học có giám sát). Nếu không có nhãn (giá trị cầu thủ), máy tính không thể học được. Dữ liệu này vô dụng cho việc train. Vậy nên ta cần xóa những dòng thiếu target
3. Lọc cầu thủ không thi đấu (total_minutes_played > 0):
Một cầu thủ không ra sân phút nào thường không có đủ cơ sở dữ liệu để định giá chính xác, hoặc là dữ liệu lỗi/cầu thủ đã giải nghệ nên cũng cần lọc ra
3. Lọc chiều cao vô lý (<= 150cm): Loại bỏ nhiễu (outliers) do nhập liệu sai.
4. Xử lý hạn hợp đồng:
Trong bóng đá, cầu thủ hết hạn hợp đồng (tự do) thường có giá trị chuyển nhượng bằng 0 hoặc không theo quy luật thị trường thông thường. Vì thế nên ta cũng xóa các cầu thủ này

In [3]:
miss_vals = ["Missing", "missing", "Unknown", "unknown", "N/A", "n/a", "?", ""]
df.replace(miss_vals, np.nan, inplace=True)

# Xử lý Target rỗng
df = df.dropna(subset=['market_value_in_eur'])

# Xử lý cầu thủ không thi đấu
df = df[df['total_minutes_played'] > 0]

# Xóa chiều cao vô lý 
df = df[df['height_in_cm'] >= 150]

# Xóa những cầu thủ có hợp đống kết thúc trước mùa giải gần nhất thi đấu
df['contract_expiration_date'] = pd.to_datetime(df['contract_expiration_date'], errors='coerce')
contract_year = df['contract_expiration_date'].dt.year
df = df[
    (contract_year >= df['last_season']) | 
    (pd.isna(contract_year))
]


print(f"Dữ liệu sau khi làm sạch: {df.shape}")

Dữ liệu sau khi làm sạch: (21422, 20)


## Xử lý missing values

Mô hình Machine Learning (đặc biệt là Sklearn) không chấp nhận ô trống (NaN) nên chúng ta cần xử lý các giá trị thiếu

* **Với biến số (Numerical):** Điền bằng Median (Trung vị).

Dùng Median tốt hơn Mean (Trung bình) vì nó ít bị ảnh hưởng bởi các giá trị ngoại lai (outliers). Ví dụ: Lương cầu thủ ngôi sao cực cao sẽ kéo Mean lên, làm sai lệch việc điền dữ liệu cho cầu thủ thường.

* **Với biến phân loại (Categorical):**

foot: Điền bằng Mode (Giá trị xuất hiện nhiều nhất - thường là chân phải).

position: Xóa những dòng trống position vì đây là yếu tố quan trọng cho mô hình dự đoán. Các cầu thủ trống position cũng trống luôn sub_position nên xóa luôn là hợp lý.

sub_position: Điền có logic. Thay vì điền bừa, code nhóm theo position chính (Vd: Defender) và lấy vị trí phụ phổ biến nhất của nhóm đó (Vd: Centre-Back) để điền. Đây là cách xử lý thông minh để giữ tính nhất quán của dữ liệu.

country_of_citizenship: Nếu thiếu quốc tịch, lấy nơi sinh (country_of_birth) đắp vào. Do hầu hết cầu thủ sẽ sinh ra và thi đấu cho cũng một đội tuyển quốc gia 

Các biến phân loại khác sẽ điền "Unknown" để tránh ảnh hưởng tới dữ liệu

In [4]:
# --- 1. Cột dạng Số (Numerical) ---
# Điền missing bằng Median (Trung vị) để tránh ảnh hưởng bởi outlier
num_cols = df.select_dtypes(include=[np.number]).columns
for col in num_cols:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].median())

# --- 2. Cột dạng Phân loại (Categorical) ---
# Cột 'foot': Điền bằng giá trị phổ biến nhất (Mode)
if df['foot'].isnull().sum() > 0:
    df['foot'] = df['foot'].fillna(df['foot'].mode()[0])

# Cột 'sub_position': Điền logic dựa theo 'position' cha
# Tạo từ điển map: {Position -> Mode của Sub-position đó}
pos_map = df.groupby('position')['sub_position'].agg(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown').to_dict()

# Áp dụng map vào những dòng bị thiếu sub_position
df['sub_position'] = df.apply(
    lambda row: pos_map.get(row['position'], 'Unknown') if pd.isna(row['sub_position']) else row['sub_position'],
    axis=1
)

# Xóa những cầu thủ thiếu thông tin vị trí, vì đây là thông tin cốt lõi không thể thay thế
df = df.dropna(subset=['position'])

df['country_of_citizenship'] = df['country_of_citizenship'].fillna(df['country_of_birth'])
df['country_of_citizenship'] = df['country_of_citizenship'].fillna('Unknown')

# Các cột categorical còn lại: Điền "Unknown"
cat_cols = df.select_dtypes(include=['object']).columns
df[cat_cols] = df[cat_cols].fillna('Unknown')

print("Đã xử lý xong Missing Values.")

Đã xử lý xong Missing Values.


### Feature engineering
**has_agent (Có người đại diện không):**

- Cầu thủ có người đại diện (agent) thường được định giá cao hơn và thương lượng lương tốt hơn. 
- Biến này được tạo bằng cách kiểm tra cột aggent_name. Nếu agent_name là "Unknown" sẽ được đánh dấu là không có agent và ngược lại

**ga_per90min (Hiệu suất bàn thắng + kiến tạo mỗi 90p):**

- So sánh tổng bàn thắng là không công bằng giữa một người đá 30 trận và người đá 5 trận. Quy về hiệu suất "mỗi 90 phút" giúp chuẩn hóa dữ liệu để so sánh năng lực thực sự.
- ga_per90min được tính bằng công thức 
  * Với cầu thủ thi đấu trên 90 phút/mùa ('total_goals' + 'total_assists')/ 'total_minutes_played' * 90
  * Với cầu thủ thi đấu dưới 90 phút/mùa ga_per90 min được tính bằng đúng  'total_goals' + 'total_assists'

**contract_years_remaining (Số năm hợp đồng còn lại):**

* Trong bóng đá, hợp đồng càng dài thì giá trị chuyển nhượng càng cao (phí phá vỡ hợp đồng). Hợp đồng sắp hết hạn giá sẽ giảm sâu.
* contract_years_remaining được tính bằng năm và bằng contract_expiration_date' - 'last_season'

**is_champion (Vô địch), is_top_4 (Trong top4), is_relegation (Xuống hạng):**

* Vị thế của CLB ảnh hưởng lớn đến giá cầu thủ. Cầu thủ đội vô địch sẽ đắt giá hơn cầu thủ đội xuống hạng. Mặc dù đã có cột club_position nhưng nhìn vào khoảng cách giữa các số là như nhau. Ví dụ: khoảng cách giữa dội thứ 3 và 4 giống như giữa đội 4 và 5. Tuy nhiên đội 3 và 4 sẽ được thi đấu cúp châu lục còn đội thứ 5 thì không. Vậy nên cần thêm 1 biến mới

* Đội có "position" = 1 sẽ được đánh dấu là champion. "position" <= 4 sẽ được đánh dấu "is_top_4" là True. Các đội xuống hạng có postion >= 18 sẽ đánh dấu "is_relegation" là True

In [5]:

# 1. Has Agent
df['has_agent'] = df['agent_name'].apply(lambda x: 0 if x == 'Unknown' else 1)

# 2. Goals + Assists per 90 min (Logic xử lý cầu thủ đá ít < 90p)
df['ga_per90min'] = np.where(
    df['total_minutes_played'] < 90,
    df['total_goals'] + df['total_assists'], 
    (df['total_goals'] + df['total_assists']) / df['total_minutes_played'] * 90
)
df['ga_per90min'] = df['ga_per90min'].round(2)

# 3. Thời hạn hợp đồng (Logic mới: Năm hết hạn - Mùa giải trước)
df['contract_expiration_date'] = pd.to_datetime(df['contract_expiration_date'], errors='coerce')
df['contract_years_remaining'] = df['contract_expiration_date'].dt.year - df['last_season']
median_years = df['contract_years_remaining'].median()
df['contract_years_remaining'] = df['contract_years_remaining'].fillna(median_years)

df['contract_years_remaining'] = df['contract_years_remaining'].astype(int)
# 4. Đội nằm trong nhóm nào
df['is_champion'] = (df['club_position'] == 1).astype(int)       
df['is_top_4'] = (df['club_position'] <= 4).astype(int)        
df['is_relegation'] = (df['club_position'] >= 18).astype(int)

print("Đã tạo xong các Feature (Contract Years = Expiry Year - Last Season).")
print(df[['last_season', 'contract_expiration_date', 'contract_years_remaining']].head())

Đã tạo xong các Feature (Contract Years = Expiry Year - Last Season).
   last_season contract_expiration_date  contract_years_remaining
0         2015                      NaT                         3
1         2017                      NaT                         3
4         2017                      NaT                         3
7         2015                      NaT                         3
9         2015               2023-12-31                         8


## Lưu dữ liệu để sử dụng cho trả lời câu hỏi

In [6]:
df.to_csv('../data/processed_players_data.csv', index=False)

### Encoding
Vì model không thể hiểu được các biến phân loại nên ở đây ta cần phải mã hóa các biến phân loại để model có thể hiểu. Ở đây chia các biến phân loại thành 2 cách ecoding

* Các cột phân loại ít giá trị: sub_position', 'foot' sử dụng one-hot encoding. Các biến này không có thứ tự tự nhiên. Chân trái không "lớn hơn" hay "nhỏ hơn" chân phải. Nếu ta gán số (0, 1, 2) như Label Encoding, mô hình có thể hiểu nhầm là có thứ tự (2 > 1). One-Hot tách chúng thành các cột riêng biệt (True/False) giúp mô hình đối xử công bằng với mọi giá trị. Ngoài ra, vì số lượng giá trị ít, việc tạo thêm vài chục cột mới không gây áp lực lớn lên bộ nhớ hay tốc độ tính toán.

* Các cột phân loại nhiều giá trị: 'current_club_name', 'country_of_citizenship', 'current_club_domestic_competition_id' sử dụng target encoding. Các cột này nói về xuất thân của cầu thủ. Nó thường mang yếu tố thứ tự. Ví dụ một cầu thủ quốc tịch Anh sẽ thường được định giá cao hơn 1 cầu thủ Việt Nam. Phương pháp này giúp chuyển đổi từ biến định danh sang biến số có quan hệ trực tiếp với bài toán. Mô hình sẽ dễ dàng học được quy luật: Giá trị đại diện (Mean Target) càng cao -> Giá trị dự đoán càng cao.

In [7]:
numeric_features = [
    'age_at_last_season', 'height_in_cm', 'club_position',
    'total_goals', 'total_assists', 'total_minutes_played',
    'total_yellow_cards', 'total_red_cards', 'has_agent',
    'ga_per90min', 'contract_years_remaining', 'last_season', 
    'is_champion', 'is_top_4', 'is_relegation'
]

# Các cột phân loại ít giá trị (Low Cardinality) -> Sẽ dùng One-Hot/Label
categorical_features_low = [
    'sub_position', 'foot'
]

# Các cột phân loại nhiều giá trị (High Cardinality) -> Sẽ dùng Target Encoding
categorical_features_high = [
    'current_club_name', 'country_of_citizenship', 'current_club_domestic_competition_id'
]

target = 'market_value_in_eur'

# Tạo dataframe gọn nhẹ
all_features = numeric_features + categorical_features_low + categorical_features_high
df_model = df[all_features + [target]].copy()

# Xử lý nhanh các giá trị null trong cột số (nếu có) bằng 0 hoặc trung bình
df_model[numeric_features] = df_model[numeric_features].fillna(0)

print("Kích thước dữ liệu ban đầu:", df_model.shape)
df_model.head()

Kích thước dữ liệu ban đầu: (21378, 21)


,age_at_last_season,height_in_cm,club_position,total_goals,total_assists,total_minutes_played,total_yellow_cards,total_red_cards,has_agent,ga_per90min,...,last_season,is_champion,is_top_4,is_relegation,sub_position,foot,current_club_name,country_of_citizenship,current_club_domestic_competition_id,market_value_in_eur
0,37.00,184.00,8.00,12,8,2429,6,0,1,0.74,...,2015,0,0,0,Centre-Forward,right,Società Sportiva Lazio S.p.A.,Germany,serie-a,"1,000,000.00"
1,37.00,190.00,4.00,0,0,181,0,0,1,0.00,...,2017,0,1,0,Goalkeeper,left,Borussia Dortmund,Germany,bundesliga,"750,000.00"
4,36.00,194.00,12.00,0,0,450,0,0,1,0.00,...,2017,0,0,0,Goalkeeper,right,FC Bayern München,Germany,bundesliga,"100,000.00"
7,35.00,179.00,2.00,2,2,679,1,0,0,0.53,...,2015,0,1,0,Attacking Midfield,both,Arsenal Football Club,Czech Republic,premier-league,"350,000.00"
9,34.00,193.00,8.00,1,0,221,0,0,0,0.41,...,2015,0,0,0,Centre-Forward,right,Málaga CF,Paraguay,laliga,"250,000.00"


### One-hot encoding
Xử lý các biến phân loại ít giá trị, không phân thứ tự (sub_position, foot) bằng One-Hot Encoding. Chúng ta làm bước này trước khi chia tập train/test vì nó không gây rò rỉ dữ liệu

In [8]:
df_encoded = pd.get_dummies(df_model, columns=categorical_features_low, drop_first=True)

print("Kích thước sau khi One-Hot Encoding:", df_encoded.shape)
df_encoded.head()

Kích thước sau khi One-Hot Encoding: (21378, 33)


,age_at_last_season,height_in_cm,club_position,total_goals,total_assists,total_minutes_played,total_yellow_cards,total_red_cards,has_agent,ga_per90min,...,sub_position_Goalkeeper,sub_position_Left Midfield,sub_position_Left Winger,sub_position_Left-Back,sub_position_Right Midfield,sub_position_Right Winger,sub_position_Right-Back,sub_position_Second Striker,foot_left,foot_right
0,37.00,184.00,8.00,12,8,2429,6,0,1,0.74,...,False,False,False,False,False,False,False,False,False,True
1,37.00,190.00,4.00,0,0,181,0,0,1,0.00,...,True,False,False,False,False,False,False,False,True,False
4,36.00,194.00,12.00,0,0,450,0,0,1,0.00,...,True,False,False,False,False,False,False,False,False,True
7,35.00,179.00,2.00,2,2,679,1,0,0,0.53,...,False,False,False,False,False,False,False,False,False,False
9,34.00,193.00,8.00,1,0,221,0,0,0,0.41,...,False,False,False,False,False,False,False,False,False,True


### Chia train test và log transform cho target
Chia dữ liệu trước khi làm Target Encoding để tránh lỗi data leakage. Đồng thời, áp dụng `np.log1p` cho biến mục tiêu để xử lý vấn đề lệch dữ liệu (skewness) của giá trị cầu thủ.

In [ ]:
train_idx, test_idx = train_test_split(df.index, test_size=0.2, random_state=42)


X_train_cat = df_model.loc[train_idx, all_features]
y_train     = np.log1p(df_model.loc[train_idx, target])

X_test_cat  = df_model.loc[test_idx, all_features] 
y_test      = np.log1p(df_model.loc[test_idx, target])

# Lưu file CatBoost 
train_cat_full = X_train_cat.copy()
train_cat_full['market_value_log'] = y_train
train_cat_full.to_csv('../data/train_catboost.csv', index=False)

test_cat_full = X_test_cat.copy()
test_cat_full['market_value_log'] = y_test
test_cat_full.to_csv('../data/test_catboost.csv', index=False)

features_enc= [col for col in df_encoded.columns if col != target]

X_train_enc = df_encoded.loc[train_idx, features_enc]
X_test_enc  = df_encoded.loc[test_idx, features_enc]

print("\nĐã chuẩn bị dữ liệu catboost xong!")



Đã chuẩn bị dữ liệu catboost xong!
------------------------------


### Target encoding
Tính trung bình giá trị (đã log) của biến mục tiêu cho từng nhóm trong tập Train, sau đó map sang tập Test. Những giá trị lạ ở tập Test (chưa từng thấy ở Train) sẽ được điền bằng giá trị trung bình toàn cục.

In [10]:
# Hàm Target Encoding đơn giản và hiệu quả
def apply_target_encoding(train_df, test_df, target_series, cols_to_encode):
    # Tạo copy để không ảnh hưởng dữ liệu gốc
    train_encoded = train_df.copy()
    test_encoded = test_df.copy()
    
    # Tạo dataframe tạm nối X_train và y_train để tính toán
    temp_train = train_df.copy()
    temp_train['target_temp'] = target_series
    
    # Tính trung bình toàn cục (để điền cho các giá trị lạ ở tập test)
    global_mean = target_series.mean()
    
    for col in cols_to_encode:
        # Tính trung bình target cho mỗi giá trị trong cột
        encoding_map = temp_train.groupby(col)['target_temp'].mean()
        
        # Map vào tập train và test
        train_encoded[col] = train_encoded[col].map(encoding_map)
        test_encoded[col] = test_encoded[col].map(encoding_map)
        
        # Xử lý các giá trị bị thiếu (do xuất hiện ở test nhưng không có ở train)
        train_encoded[col] = train_encoded[col].fillna(global_mean)
        test_encoded[col] = test_encoded[col].fillna(global_mean)
        
    return train_encoded, test_encoded

# Áp dụng hàm
X_train_final, X_test_final = apply_target_encoding(
    X_train_enc, 
    X_test_enc, 
    y_train, 
    categorical_features_high
)

print("Đã hoàn thành Target Encoding!")
X_train_final[categorical_features_high].head()

Đã hoàn thành Target Encoding!


,current_club_name,country_of_citizenship,current_club_domestic_competition_id
17364,13.38,13.41,13.67
15693,12.23,13.09,12.98
25262,14.68,13.06,13.52
1184,12.26,12.95,12.78
27589,13.01,14.44,12.78


### Lưu tập train/test

In [ ]:

train_data = X_train_final.copy()
train_data['market_value_log'] = y_train

test_data = X_test_final.copy()
test_data['market_value_log'] = y_test

train_data.to_csv('../data/train_data_encoded.csv', index=False)
test_data.to_csv('../data/test_data_encoded.csv', index=False)

print("Đã lưu xong:")
print("- train_data_encoded.csv")
print("- test_data_encoded.csv")

Đã lưu xong:
- train_data_encoded.csv
- test_data_encoded.csv
